# Running a simple control-loop with one Controller

## After  completing this tutorial you will be able to invoke a simple control-loop which regulates a trafo-tap-changer

For introduction purposes an easy example will be described.
The task at hand would be to simulate a Trafo Controller with local continous tap changer voltage control. First we have to load all necesary packages. Of course we need the pandapower-package but also the control module and 
the network module.

We load the network and pass it to the parameter net. A first powerflow gives us results. We have a look at the trafo-resulttable. I wrote ``import pandapower as pp`` which provides me a handy abbreviation ``pp`` for the whole import-reference. These abbreviations have to be unique throughout your code.

In [6]:
# Importing necessary packages
import pandapower as pp
import control
from pandapower.networks import mv_oberrhein
# loading the network with the usecase 'generation'
net = mv_oberrhein()
pp.runpp(net)

# checking the results
print(net.res_trafo)

       p_hv_mw  q_hv_mvar    p_lv_mw  q_lv_mvar     pl_mw   ql_mvar   i_hv_ka  \
114  17.270680   3.955948 -17.207125  -2.635336  0.063556  1.320612  0.092995   
142  20.863017   4.653035 -20.784893  -2.789431  0.078123  1.863605  0.112193   

      i_lv_ka  vm_hv_pu  va_hv_degree  vm_lv_pu  va_lv_degree  loading_percent  
114  0.495289       1.0           0.0  1.014598     -4.212528        70.871819  
142  0.588438       1.0           0.0  1.028804     -4.943953        85.502393  


To show a simple control-behavior, we define an instance of **ContiniousTapControl** for the tranformer with the ID 114 and pass the voltagesetpoint 0.98 p.u. with a small tolerance of 1e-6.

In [7]:
# initialising controller
tol = 1e-6
trafo_controller = control.ContinuousTapControl(net=net, tid=114, u_set=0.98, tol=tol)

We imported **pandapower** and the **control** module and created the object of a controller we need. You can look up which
parameters are mandatory and which are optional in the constructor of the class you are creating
an instance of. In our example we need to pass a reference to the net, the ID of the controlled
transformer, the voltage setpoint and a calculation tolerance. Now we look at our network that contains our controller


In [8]:
net.controller

,controller,in_service,order,level,recycle
0,ContinuousTapControl of trafo 114,True,0.0,0,False


The output in the console shows, that the controller is active and has the default values for order and level (we'll look at 
these in more detail shortly). We invoke the control-loop by setting **run_control=True** within the loadflow arguments.

In [10]:
# running a control-loop
pp.runpp(net, run_control=True)

In [5]:
# checking the results
print(net.res_trafo)

       p_hv_mw  q_hv_mvar    p_lv_mw  q_lv_mvar     pl_mw   ql_mvar   i_hv_ka  \
114  17.299985   4.185280 -17.235828  -2.762132  0.064157  1.423147  0.093421   
142  20.863017   4.653035 -20.784893  -2.789431  0.078123  1.863605  0.112193   

      i_lv_ka  vm_hv_pu  va_hv_degree  vm_lv_pu  va_lv_degree  loading_percent  
114  0.514188       1.0           0.0  0.980000     -4.513536        71.247940  
142  0.588438       1.0           0.0  1.028804     -4.943953        85.502393  


After comparing both resulttables it is clearly shown, that the voltage at the controlled transformer has changed from 1.014598 p.u. to the defined voltagesetpoint 0.98 p.u. 